In [2]:
import numpy as np
import boule as bl

In [3]:
ellipsoid = bl.GRS80

In [21]:
lat = 5/3600 
h = 2850

In [33]:
b = ellipsoid.semiminor_axis
a = ellipsoid.semimajor_axis
E = ellipsoid.linear_eccentricity
Gm = ellipsoid.geocentric_grav_const
omega = ellipsoid.angular_velocity

print(f"E={E}")
print(f"a={a}")
print(f"b={b}")
print(f"Gm={Gm}")
print(f"omega={omega}")

E=521854.009700248
a=6378137
b=6356752.314140356
Gm=398600500000000.0
omega=7.292115e-05


In [25]:
beta = np.rad2deg(np.arctan(b*np.sin(np.deg2rad(lat))/a*np.cos(np.deg2rad(lat))))
print(f"beta={beta}")

beta=0.0013842322065756726


In [26]:
# Compute z_p^2
zp_squared = (b * np.sin(np.deg2rad(beta)) + h * np.sin(np.deg2rad(lat)))**2

# Compute r_p^2
rp_squared = (a * np.cos(np.deg2rad(beta)) + h * np.cos(np.deg2rad(lat)))**2

print("z_p^2 =", zp_squared)
print("r_p^2 =", rp_squared)


z_p^2 = 23606.62283126165
r_p^2 = 40716995070403.35


In [27]:
# Cálculo de D e R
D = (rp_squared - zp_squared) / E**2
R = (rp_squared + zp_squared) / E**2

print("D =", D)
print("R =", R)

D = 149.51255724414438
R = 149.51255741751115


In [46]:

# Calcula cos²(beta')
cos2_beta_p = 0.5 + 0.5 * R - np.sqrt(0.25 + 0.25 * R**2 - 0.5 * D)

sin2_beta_p = 1 - cos2_beta_p



# Garante que o valor esteja no intervalo [0, 1] por segurança numérica
# cos2_beta_p = np.clip(cos2_beta_p, 0, 1)

# Calcula beta' em radianos
beta_p = np.arccos(np.sqrt(cos2_beta_p))

# Se quiser em graus:
beta_p_deg = np.rad2deg(beta_p)

print("cos²(beta') =", cos2_beta_p)
print("sen²(beta') =", sin2_beta_p)
print("beta' (rad) =", beta_p)
print("beta' (graus) =", beta_p_deg)


cos²(beta') = 0.9999999994163176
sen²(beta') = 5.836824357174919e-10
beta' (rad) = 2.4159520602567474e-05
beta' (graus) = 0.0013842385655864757


In [29]:
# Cálculo de b'
b_p = np.sqrt(rp_squared + zp_squared - E**2 * cos2_beta_p)

print("b' =", b_p)


b' = 6359611.897492543


In [30]:

# Termos individuais
term1 = 1 + 3 * (b**2 / E**2)
term2 = np.arctan(E / b)
term3 = 3 * (b / E)

# Resultado final
q_zero = 0.5 * (term1 * term2 - term3)

print(f"q_zero: {q_zero}")

q_zero: 7.334625840726972e-05


In [31]:
# Calculate the expression
term1 = 1 + (b_p**2 / E**2)
term2 = 1 - (b_p / E) * np.arctan(E / b_p)
q_p = 3 * term1 * term2 - 1

print("q_p:", q_p)


q_p: 0.002685631457639115


In [50]:
# T_1
T1 = Gm / (b_p**2 + E**2)

# T_2
T2 = (omega**2 * a**2 * E * q_p) / ((b_p**2 + E**2) * q_zero)

# T_3
T3 = (sin2_beta_p / 2) - (1/6)

# T_4
T4 = omega**2 * b_p * cos2_beta_p

# Impressão dos resultados
print("T1 =", T1)
print("T2 =", T2)
print("T3 =", T3)
print("T4 =", T4)


T1 = 9.789536263128698
T2 = 0.10151645913370078
T3 = -0.16666666637482544
T4 = 0.033817198833632274


In [52]:
# Termos auxiliares
numerator = b_p**2 + E**2 * sin2_beta_p**2
denominator = b_p**2 + E**2

# Expressão final
W = np.sqrt(numerator / denominator)

print("W=", W)


W= 0.99665018867654


In [53]:
# Cálculo de gamma
gamma = (T1 + (T2*T3) - T4) / W

print("gamma =", gamma)


gamma = 9.771532444499215


In [54]:
gamma = bl.GRS80.normal_gravity(latitude=lat, height=h)
print(f"{gamma:.2f} mGal")

977153.24 mGal
